In [22]:
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.warehouse import Warehouse

import pandas as pd

In [23]:
## Create a session using our config file
session = Session.builder.config("connection_name","myconnection").create()
root = Root(session)

In [24]:
## Create a New or Update an Existing Warehouse
investment_wh = Warehouse(
  name="investment_wh",
    warehouse_size="SMALL",
    auto_suspend=300,
    auto_resume='true'
)
investment_wh_res = root.warehouses["investment_wh"]
investment_wh_res.create_or_update(investment_wh)

In [25]:
## Create or replace Database in the account
database = root.databases.create(Database(name="INVESTMENT_DB"), mode="orreplace")

In [26]:
## Create or replace Schema in a Database
schema = database.schemas.create(Schema(name="INVESTMENT_SCHEMA"), mode="orreplace")

### Load sp500_returns dataset into Snowflake

In [27]:
## Load in our returns CSV file to a pandas DataFrame
sp500_returns_pd =  pd.read_csv("../csv_dumps/sp500_returns.csv")
sp500_returns_pd.tail()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
250636,2021-12-27,ZTS,242.502472,246.509995,246.720001,243.080002,244.389999,807000.0
250637,2021-12-28,ZTS,240.279221,244.250000,247.839996,243.639999,247.000000,1004400.0
250638,2021-12-29,ZTS,243.014023,247.029999,247.850006,243.639999,244.020004,939900.0
250639,2021-12-30,ZTS,241.174454,245.160004,249.270004,244.960007,247.050003,859100.0
250640,2021-12-31,ZTS,240.062805,244.029999,246.990005,243.740005,245.160004,1073400.0


In [28]:
## Create a Snowpark DataFrame object from the pandas DataFrame
sp500_returns_df = session.create_dataframe(sp500_returns_pd)
sp500_returns_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------
|"Date"      |"Symbol"  |"Adj Close"        |"Close"            |"High"             |"Low"              |"Open"             |"Volume"   |
-----------------------------------------------------------------------------------------------------------------------------------------
|2020-01-02  |A         |83.77913665771484  |85.94999694824219  |86.3499984741211   |85.19999694824219  |85.9000015258789   |1410500.0  |
|2020-01-03  |A         |82.43399047851562  |84.56999969482422  |85.33000183105469  |84.5               |84.66999816894531  |1118300.0  |
|2020-01-06  |A         |82.67767333984375  |84.81999969482422  |84.81999969482422  |83.5999984741211   |84.0               |1993200.0  |
|2020-01-07  |A         |82.93112182617188  |85.08000183105469  |85.26000213623047  |83.94000244140625  |83.95999908447266  |1684700.0  |
|2020-01-08  |A         |83.749893

In [29]:
## Save the Snowpark DataFrame to a Snowflake table in our account
sp500_returns_df.write.mode("overwrite").save_as_table("sp500_returns")

### Load sp500_info dataset into Snowflake

In [30]:
## Load in our info CSV file to a pandas DataFrame
sp500_info_pd = pd.read_csv("../csv_dumps/sp500_info.csv")
sp500_info_pd.tail()

,symbol,address1,city,state,zip,country,phone,website,industry,industryKey,...,pegRatio,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationMean,numberOfAnalystOpinions,earningsGrowth,fax,industrySymbol
498,YUM,1441 Gardiner Lane,Louisville,KY,40213,United States,502 874 8300,https://www.yum.com,Restaurants,restaurants,...,1.73,160.0,116.0,138.10,136.0,2.4,21.0,0.281,NaN,NaN
499,ZBRA,3 Overlook Point,Lincolnshire,IL,60069,United States,847 634 6700,https://www.zebra.com,Communication Equipment,communication-equipment,...,5.21,340.0,183.0,265.67,275.0,2.4,9.0,NaN,NaN,NaN
500,ZBH,345 East Main Street,Warsaw,IN,46580,United States,574 373 3121,https://www.zimmerbiomet.com,Medical Devices,medical-devices,...,2.27,150.0,99.0,129.38,130.0,2.6,24.0,-0.163,NaN,NaN
501,ZION,One South Main Street,Salt Lake City,UT,84133-1109,United States,801 844 7637,https://www.zionsbancorporation.com,Banks - Regional,banks-regional,...,-0.26,45.0,29.0,36.60,37.0,2.7,20.0,-0.191,NaN,NaN
502,ZTS,10 Sylvan Way,Parsippany,NJ,07054,United States,973 822 7000,https://www.zoetis.com,Drug Manufacturers - Specialty & Generic,drug-manufacturers-specialty-generic,...,3.38,255.0,190.0,216.18,215.0,1.9,11.0,0.142,NaN,NaN


In [32]:
## Create a Snowpark DataFrame object from the pandas DataFrame
sp500_info_df = session.create_dataframe(sp500_info_pd)
sp500_info_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
## Save the Snowpark DataFrame to a Snowflake table in our account
sp500_info_df.write.mode("overwrite").save_as_table("sp500_info")